In [7]:
from rdflib import Graph
from rdflib.plugins.stores.sparqlstore import SPARQLStore

endpoint_url = "https://data.cimple.eu/sparql"  

store = SPARQLStore(endpoint=endpoint_url)
g = Graph(store)

query = """
SELECT DISTINCT count(?s) as ?nb ?type
WHERE {
  ?s a ?type .
}
GROUP BY ?type
ORDER BY DESC(?nb)
"""

results = g.query(query)
for row in results:
    print(row)

SPARQLConnectorException: Query endpoint not set!